# BAN 5753 Ensemble Learning Exercise


In [1]:
# import all necessary libraries
import pandas as pd
import os
import numpy as np
from numpy import mean
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import shap
import warnings
warnings.filterwarnings('ignore')
from pylab import rcParams
#rcParams['figure.figsize']=16,15
rcParams.update({'font.size': 14})
# pd.options.display.max_columns = 25
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from numpy import argmax
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, average_precision_score

c:\Users\cwolt\OneDrive\Desktop\Data-Science-Projects\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Preprocessing

In [12]:
# load data set
df = pd.read_csv("C:\\Users\\cwolt\\OneDrive\\Desktop\\BAnDS\\Fall 2025\\Advanced Business Analytics\\Module 3\\HW\\hospital_admissions_with_missing.csv")
print(df.head())
print(df.dtypes)

   Admitted  avg_income  CLINIC_VISIT REFERRAL_SOURCE  Visit_Date Visit_Month  \
0         0    46045.39             0              ER           4         Jan   
1         0    19145.43             1            Self           4         Sep   
2         0    48816.69             0            Self           9         Oct   
3         0    68702.11             0              ER           8         Mar   
4         1    63064.45             0            Self           2         Sep   

   Visit_Year  distance_to_hospital ETHNICITY   BMI  Patient_ID  \
0        2024                 11.40     Asian  35.2       10001   
1        2025                  5.20     White  26.3       10002   
2        2023                  2.67       NaN  25.2       10003   
3        2024                 13.27     White  26.9       10004   
4        2024                 14.49     Black  37.9       10005   

   symptom_duration Local_Resident    BP_Score  Pulse_Score  \
0                 9              Y  120.417310 

In [13]:
# Drop unnecessary variables (Visit_Year, Patient_ID, PRIOR_PROVIDER_TYPE, TOTAL, AllocProportion, SampleSize, ActualProportion, SelectionProb, and SamplingWeight)
df = df.drop(columns=['Visit_Year', 'Patient_ID', 'PRIOR_PROVIDER_TYPE', 'TOTAL', 'AllocProportion', 'SampleSize', 'ActualProportion', 'SelectionProb', 'SamplingWeight'])
print(df.head())

   Admitted  avg_income  CLINIC_VISIT REFERRAL_SOURCE  Visit_Date Visit_Month  \
0         0    46045.39             0              ER           4         Jan   
1         0    19145.43             1            Self           4         Sep   
2         0    48816.69             0            Self           9         Oct   
3         0    68702.11             0              ER           8         Mar   
4         1    63064.45             0            Self           2         Sep   

   distance_to_hospital ETHNICITY   BMI  symptom_duration Local_Resident  \
0                 11.40     Asian  35.2                 9              Y   
1                  5.20     White  26.3                 6              Y   
2                  2.67       NaN  25.2                 8              Y   
3                 13.27     White  26.9                 5              Y   
4                 14.49     Black  37.9                13              N   

     BP_Score  Pulse_Score  interest_in_admission INSURA

In [ ]:
# Encode Cyclical Features – Visit_Date and Visit _Month.
df['Visit_Date'] = pd.to_datetime(df['Visit_Date'])
df['Visit_Month'] = df['Visit_Date'].dt.month
df['Visit_Year'] = df['Visit_Date'].dt.year
df['Visit_Day'] = df['Visit_Date'].dt.day
df['Visit_Month'] = df['Visit_Month'].apply(lambda x: np.sin(2 * np.pi * x / 12))
df['Visit_Day'] = df['Visit_Day'].apply(lambda x: np.sin(2 * np.pi * x / 31))

In [ ]:
# Encode Local_Resident column to integer Values.
df['Local_Resident'] = df['Local_Resident'].astype('category').cat.codes

In [14]:
# Extra Credit: Handle the missing values in the dataset
# Check for missing values
print(df.isnull().sum())
# Fill missing values with appropriate strategies
df['Local_Resident'] = df['Local_Resident'].fillna(df['Local_Resident'].median())
df['Visit_Month'] = df['Visit_Month'].fillna(df['Visit_Month'].median())
df['Visit_Day'] = df['Visit_Day'].fillna(df['Visit_Day'].median())

Admitted                   0
avg_income                75
CLINIC_VISIT               0
REFERRAL_SOURCE           60
Visit_Date                 0
Visit_Month                0
distance_to_hospital       0
ETHNICITY                 60
BMI                       75
symptom_duration           0
Local_Resident             0
BP_Score                   0
Pulse_Score                0
interest_in_admission      0
INSURANCE_TYPE             0
FOLLOWUP_EMAILS            0
REFERRED_BY_SPECIALIST     0
REFERRAL_CONTACTS          0
OXYGEN_LEVEL              75
PATIENT_CALLS              0
sex                        0
HOSPITAL_REACHOUTS         0
TELEMEDICINE_SCORE         0
SERVICE_REGION             0
TOTAL_CONTACTS             0
TRAVEL_REQUESTS            0
dtype: int64


TypeError: Cannot convert ['Y' 'Y' 'Y' ... 'Y' 'Y' 'N'] to numeric